In [1]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()
#import Pkg; Pkg.activate(joinpath(@__DIR__,"..")); Pkg.instantiate()

  Activating environment at `~/Desktop/PHD2022/Autumn/CSE 579/CS_project/LQ_Game_Solver/CS_project/Project.toml`


In [2]:
using LinearAlgebra
using Plots
using SparseArrays
using ForwardDiff
#using ControlSystems

### 2D Point Mass Continous dynamics

In [3]:
c = 0.0
m = 1.0
function point_mass(x, u)
   # x = state[1]                #x_1 = ẋ
    ẋ = x[3]
    ẍ = -(c/m)*ẋ + u[1]/(m)  #x_2 = ẍ = (-c/m)ẋ + u/m 
    #y = state[3]
    ẏ = x[4]
    ÿ = -(c/m)*ẏ + u[2]/(m)  #x_2 = ẍ = (-c/m)ẋ + u/m 
    return [ẋ; ẏ; ẍ; ÿ]
end

point_mass (generic function with 1 method)

### Linearizing and discretizing the dynamics

In [4]:
function lin_dyn_discrete(dynamics, x, u, dt)
    A = ForwardDiff.jacobian(dx -> dynamics(dx, u), x)
    B = ForwardDiff.jacobian(du -> dynamics(x, du), u)
    A = dt .* A + I
    B = dt .* B
    return A, B
end
x = [0; 0; 0 ;0]
u = [0; 0]
A, B = lin_dyn_discrete(point_mass, x, u, 0.01)

4×2 Matrix{Float64}:
 0.0   0.0
 0.0   0.0
 0.01  0.0
 0.0   0.01

In [43]:
Q = sparse(zeros(4,4)) #state cost for point mass 1
Q[1,1] = 1; Q[2,2] = 1; Q[3,3] = 1; Q[4,4] = 1;
R = I(2)
function cost(x, u)
    c = 0.5*x'*Q*x + 0.5*u'*R*u #+ max(0, (x[1] - x[3])^2)
    return c
end

cost (generic function with 1 method)

In [44]:
x = [0; 0; 1; 0]
u = [1; 3]
cost(x, u)

5.5

In [45]:
#Q = ForwardDiff.hessian(dx -> cost(dx, u), x)
R = ForwardDiff.hessian(du -> cost(x, du), u)
l = ForwardDiff.gradient(dx -> cost(dx, u), x)

4-element Vector{Float64}:
 0.0
 0.0
 1.0
 0.0

In [46]:
function quadratic_cost(cost_func, x, u)
    """
    2nd order Taylor expansion of cost at t
    I neglected the mixed paritals in the hessian
    """
    Q = ForwardDiff.hessian(dx -> cost_func(dx, u), x)
    l = ForwardDiff.gradient(dx -> cost_func(dx, u), x)
    R = ForwardDiff.hessian(du -> cost_func(x, du), u)
    r = ForwardDiff.gradient(du -> cost_func(x, du), u)
    cost = 0.5 * x' * (Q*x + 2*l) + 0.5 * u' * (R*u + 2*r)
    return cost, Q, l, R, r
end

quadratic_cost (generic function with 1 method)

In [47]:
quadratic_cost(cost, x, u)


(16.5, [1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0], [0.0, 0.0, 1.0, 0.0], [1.0 0.0; 0.0 1.0], [1.0, 3.0])

#### Point Mass Discrete

$$ \frac{d}{dt}x = Ax + \sum Bu$$

Single 2D point mass:
$$\frac{d}{dt}\begin{bmatrix} x \\ y \\ \dot{x} \\ \dot{y}\end{bmatrix} =  
 \begin{bmatrix} 0 & 0 & 1 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & -c/m & 0 \\ 0 & 0 & 0 & -c/m \end{bmatrix}
 \begin{bmatrix} x \\ y \\ \dot{x} \\ \dot{y}\end{bmatrix} + 
 \begin{bmatrix} 0 & 0 \\ 0 & 0 \\ 1/m & 0 \\ 0 & 1/m \end{bmatrix}
 \begin{bmatrix} u_x \\ u_y\end{bmatrix} $$

two 2D point masses:
$$\frac{d}{dt}\begin{bmatrix} x_1 \\ y_1 \\ \dot{x}_1 \\ \dot{y}_1 \\ x_2 \\ y_2 \\ \dot{x}_2 \\ \dot{y}_2\end{bmatrix} =  
 \begin{bmatrix} 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\ 
                0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\ 
                0 & 0 & -c/m_1 & 0 & 0 & 0 & 0 & 0\\ 
                0 & 0 & 0 & -c/m_1 & 0 & 0 & 0 & 0\\
                0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\ 
                0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\ 
                0 & 0 & 0 & 0 & 0 & 0 & -c/m_2 & 0\\ 
                0 & 0 & 0 & 0 & 0 & 0 & 0 & -c/m_2\\ \end{bmatrix}
 \begin{bmatrix} x_1 \\ y_1 \\ \dot{x}_1 \\ \dot{y}_1 \\ x_2 \\ y_2 \\ \dot{x}_2 \\ \dot{y}_2\end{bmatrix} + 
 \begin{bmatrix} 0 & 0 \\ 0 & 0 \\ 1/m_1 & 0 \\ 0 & 1/m_1 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0\end{bmatrix}
 \begin{bmatrix} u_x^1 \\ u_y^1 \end{bmatrix} +
 \begin{bmatrix} 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 0 & 0 \\ 1/m_2 & 0 \\ 0 & 1/m_2\end{bmatrix}
 \begin{bmatrix} u_x^2 \\ u_y^2 \end{bmatrix}  $$


### State and input Jacobian of 2 point masses

In [10]:
m₁ = 1
m₂ = 1
c = 0
A1 = sparse([0 0 1 0; 0 0 0 1; 0 0 (-c/m₁) 0; 0 0 0 (-c/m₁)])
A2 = sparse([0 0 1 0; 0 0 0 1; 0 0 (-c/m₂) 0; 0 0 0 (-c/m₂)])
A = blockdiag(A1, A2)
B1 = sparse([0 0; 0 0; (1/m₁) 0; 0 (1/m₁); 0 0; 0 0; 0 0; 0 0])  #Control Jacobian for point mass 1
B2 = sparse([0 0; 0 0; 0 0; 0 0; 0 0; 0 0; (1/m₂) 0; 0 (1/m₂)])    #Control Jacobian for point mass 2

8×2 SparseMatrixCSC{Float64, Int64} with 2 stored entries:
  ⋅    ⋅ 
  ⋅    ⋅ 
  ⋅    ⋅ 
  ⋅    ⋅ 
  ⋅    ⋅ 
  ⋅    ⋅ 
 1.0   ⋅ 
  ⋅   1.0

### Discretize

In [11]:
dt = 0.01 #step size
H = 30.0  #Horizon
k_steps = trunc(Int, H/dt) 

Ad = dt .* A + I    #discretize (zero order hold)
B1d = dt .*B1   #discrete (zero order hold)
B2d = dt .*B2;   #discrete (zero order hold)
Ad

8×8 SparseMatrixCSC{Float64, Int64} with 12 stored entries:
 1.0   ⋅   0.01   ⋅     ⋅    ⋅    ⋅     ⋅ 
  ⋅   1.0   ⋅    0.01   ⋅    ⋅    ⋅     ⋅ 
  ⋅    ⋅   1.0    ⋅     ⋅    ⋅    ⋅     ⋅ 
  ⋅    ⋅    ⋅    1.0    ⋅    ⋅    ⋅     ⋅ 
  ⋅    ⋅    ⋅     ⋅    1.0   ⋅   0.01   ⋅ 
  ⋅    ⋅    ⋅     ⋅     ⋅   1.0   ⋅    0.01
  ⋅    ⋅    ⋅     ⋅     ⋅    ⋅   1.0    ⋅ 
  ⋅    ⋅    ⋅     ⋅     ⋅    ⋅    ⋅    1.0

In [12]:
Q1 = sparse(zeros(8,8)) #state cost for point mass 1
Q1[1,1] = 1; Q1[2,2] = 1; Q1[3,3] = 1; Q1[4,4] = 1; Q1[5,1] = 1; Q1[5,5] = -1; Q1[6,1]=1; Q1[6,6]=-1;
Q2 = sparse(zeros(8,8))   #state cost for point mass 2
Q2[5,5] = 1; Q2[6,6] = 1; Q2[7,7] = 1; Q2[8,8] = 1;
R11 = I(2)    #Control cost for player 1
R22 = I(2)    #Contorl cost for player 2
R12 = sparse(zeros(2,2))    #Control cost for player 1 associated with player 2's controls
R21 = sparse(zeros(2,2))    #Control cost for player 2 associated with player 1's controls


2×2 SparseMatrixCSC{Float64, Int64} with 0 stored entries:
  ⋅    ⋅ 
  ⋅    ⋅ 

In [13]:
Q1

8×8 SparseMatrixCSC{Float64, Int64} with 8 stored entries:
 1.0   ⋅    ⋅    ⋅     ⋅     ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅     ⋅     ⋅    ⋅    ⋅ 
  ⋅    ⋅   1.0   ⋅     ⋅     ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅   1.0    ⋅     ⋅    ⋅    ⋅ 
 1.0   ⋅    ⋅    ⋅   -1.0    ⋅    ⋅    ⋅ 
 1.0   ⋅    ⋅    ⋅     ⋅   -1.0   ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅     ⋅     ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅     ⋅     ⋅    ⋅    ⋅ 

In [14]:
x0 = [0; 0; 1; 0; 0; 0; 0; 0]
xref = [2; 1; 1; 2; 0; 0; 0; 0]
function cost(xref, x0)
    x_k = copy(xref - x0)
    c1 = 0
    c2 = 0
    for k in length(k_steps)
        K1 = -sparse([0.1 0.1 0.1 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0])
        K2 = -sparse([0.3 -0.1 0.2 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0])
        u1 = K1*x_k
        u2 = K2*x_k
        x_k = Ad*x_k + B1d*u1 + B2d*u2
        c1 += x_k'*Q1*x_k + u1'*R11*u1 + u2'*R12*u2
        c2 += x_k'*Q2*x_k + u2'*R22*u2 + u2'*R21*u2
    end
    return c1,c2
end

cost (generic function with 1 method)

In [15]:
c1, c2 = cost(xref, x0)

(9.130409, 0.250025)

\begin{bmatrix} S11 & S12 \\ S21 & S22 \end{bmatrix}
\begin{bmatrix} P1 \\ P2 \end{bmatrix} = 
\begin{bmatrix} Y1 \\ Y2 \end{bmatrix}

\begin{bmatrix} R^{11} + (B^{1'} * V₁ * B^1) & (B^{1'} * V₁ * B^2) \\ (B^{2'} * V_2 * B^1) & R^{22} + (B^{2'} * V_2 * B^2) \end{bmatrix}

In [16]:
k_steps

3000

In [31]:
n = 8 #8 states
m = 2 #2 controls
function lqGame!(n, m)
    """
    Input
    Ad is an nxn matrix (8x8)
    B1d is an nxm matrix (8x2)
    B2d is an nxm matrix (8x2)
    u is mx1 (2x1)
    Q1 is an nxn matrix (8x8)
    Q2 is an nxn matrix (8x8)
    R11 is an mxm matrix (2x2)
    R12 is an mxm matrix (2x2)
    R21 is an mxm matrix (2x2)
    R22 is an mxm matrix (2x2)
    Output
    P₁ is an mxn matrix (2x8)
    P₂ is an mxn matrix (2x8)
    """

    V₁ = copy(Q1) #at last time step
    V₂ = copy(Q2) #at last time step
    P₁ = zeros(k_steps, n*m)
    P₂ = zeros(k_steps, n*m)
    #α₁ = zeros(T[1], m)
    #α₂ = zeros(T[1], m)

    for t in 1:k_steps
        # solving for Ps, check equation 19 in document
        # Player 1
        S11 = R11 + (B1d' * V₁ * B1d) #2x2
        S12 = B1d' * V₁ * B2d # 2x2
        S22 = R22 + (B2d' * V₂ * B2d)
        S21 = B2d' * V₂ * B1d 
        S = [S11 S12; S21 S22] # 4x4
        Y1 = B1d' * V₁ * Ad # 2x8
        Y2 = B2d' * V₂ * Ad 
        Y = [Y1; Y2] # 4 x 8
        P = S\Y
        P₁ₜ = P[1:2, :]
        P₂ₜ = P[3:4, :]
        # Player 2
        # Assign Values
        P₁[t, :] = reshape(P₁ₜ, (1, n*m))
        P₂[t, :] = reshape(P₂ₜ, (1, n*m))
        #Update value function(s)
        Fₜ = Ad - (B1d*P₁ₜ + B2d*P₂ₜ)
        V₁ = Q1 + (P₁ₜ' * R11 * P₁ₜ) + (P₂ₜ' * R12 * P₂ₜ) + (Fₜ' * V₁ * Fₜ)
        V₂ = Q2 + (P₁ₜ' * R21 * P₁ₜ) + (P₂ₜ' * R22 * P₂ₜ) + (Fₜ' * V₂ * Fₜ)
        #K[i, :] = reshape(transpose(K_i), (n*m,1))
        #println("f",reshape(K[i, :], (2,4))
    end
    return P₁, P₂#, α₁, α₂
end

lqGame! (generic function with 1 method)

In [90]:
n = 4 #8 states
m = 2 #2 controls
function affinelq!(Aₜ, Bₜ, Qₜ, lₜ, Rₜ , rₜ)

    V = copy(Qₜ[:,:,end]) #at last time step
    ζ = copy(lₜ[:,end]) #at last time step
    #P = zeros(k_steps, n*m)
    P = zeros(Float32, (m, n, k_steps))
    #α = zeros(k_steps, m)
    α = zeros(Float32, (m, k_steps))

    #α₂ = zeros(T[1], m)

    for t in k_steps:-1:1
        # solving for Ps, check equation 19 in document
        # Player 1
        Pₜ = (Rₜ[:,:,t] + (Bₜ[:,:,t]' * V * Bₜ[:,:,t]))\(Bₜ[:,:,t]' * V * Aₜ[:,:,t])
        αₜ = (Rₜ[:,:,t] + (Bₜ[:,:,t]' * V * Bₜ[:,:,t]))\(Bₜ[:,:,t]' * ζ)
        # Assign Values
        #P[t, :] = reshape(Pₜ, (1, n*m))
        #α[t, :] = αₜ
        P[:,:,t] = Pₜ
        α[:,t] = αₜ
        #Update value function(s)
        Fₜ = Aₜ[:,:,t] - (Bₜ[:,:,t]*Pₜ)
        βₜ = - Bₜ[:,:,t] * αₜ
        ζ = lₜ[:,t] + (Pₜ' * Rₜ[:,:,t] * αₜ - Pₜ' * rₜ[:,t]) + Fₜ'*(ζ + V * βₜ)
        V = Qₜ[:,:,t]' + (Pₜ' * Rₜ[:,:,t] * Pₜ) + (Fₜ' * V * Fₜ)
        #K[i, :] = reshape(transpose(K_i), (n*m,1))
        #println("f",reshape(K[i, :], (2,4))
    end
    return P, α
end

affinelq! (generic function with 1 method)

In [68]:
P₁, P₂ = lqGame!(n,m);

DimensionMismatch: DimensionMismatch("parent has 16 elements, which is incompatible with size (1, 8)")

In [69]:
P₁[1,:]

16-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.009999000099990002
 0.0
 0.0
 0.009999000099990002
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [34]:
reshape(P₁[1, :], (m, n))

2×8 Matrix{Float64}:
 0.0  0.0  0.009999  0.0       0.0  0.0  0.0  0.0
 0.0  0.0  0.0       0.009999  0.0  0.0  0.0  0.0

In [143]:
function Rollout_RK4(fun, x₀, xf, H, dt, P, α)
    """
    Rollout dynamics with initial state x₀ 
    and control law u = -Px - α
    P is an n x b gain matrix
    α is m x 1
    """
    m = 2 #2 controls
    k_steps = trunc(Int, H/dt) 
    xₜ = zeros(k_steps, length(x₀)) # 1500 x n
    uₜ = zeros(k_steps, m) 
    xₜ[1,:] .= x₀
    for t=1:(k_steps-1)
        uₜ[t,:] = -P[:,:,t]*(xₜ[t,:] - xf) - α[:,t]
        k1 = fun(xₜ[t,:], uₜ[t,:])
        k2 = fun(xₜ[t,:] + 0.5*dt*k1, uₜ[t,:])
        k3 = fun(xₜ[t,:] + 0.5*dt*k2, uₜ[t,:])
        k4 = fun(xₜ[t,:] + dt*k3, uₜ[t,:])
        xₜ[t+1,:] = xₜ[t,:] + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)
    end
    
    return xₜ, uₜ
end

Rollout_RK4 (generic function with 2 methods)

In [144]:
x₀=[0; 0; 0; 0]
dt = 0.01 #step size
H = 30.0  #Horizon
P = zeros(2,4,3000)
α = zeros(2,3000)

2×3000 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

In [145]:
# x0 = [1; 0; 0; 0; 0; 1; 0; 0]
# xref = [1; 2; 0; 0; 2; 1; 0; 0]
# #Forward rollout starting at x0
# xhist = zeros(k_steps,n)
# xhist[1,:] = x0
# u1_all = zeros(k_steps, m)
# u2_all = zeros(k_steps, m)
# for t = 1:(k_steps - 1)
#     #uhist[t,:] = -K[t,:]*(xhist[t,:] - xref)
#     u1_all[t,:] .= -reshape(P₁[t, :], (m, n))*(xhist[t,:] - xref)
#     u2_all[t,:] .= -reshape(P₂[t, :], (m, n))*(xhist[t,:] - xref)

#     xhist[t+1,:] .= Ad*xhist[t,:] + B1d*u1_all[t,:] + B2d*u2_all[t,:]#reshape(Bd,(n,m))*uhist[t,:]
# end
# #K

In [146]:
# anim = @animate for t in 1:k_steps
#     scatter([xhist[t, :][1]], [xhist[t, :][2]], m = (:circle, 12),
#     xlims = (-2, 3), 
#     ylims = (-1, 2), )
#     scatter!([xhist[t, :][5]], [xhist[t, :][6]], m = (:circle, 12),
#     xlims = (-2, 3), 
#     ylims = (-1, 2), )
# end every 10;
# gif(anim, "pms.gif")

In [147]:
#function ilqr()
    # 1. Initialize an initial feasible trajectory
n = 4
m = 2
x₀ = [0; 0; 0; 0]
xf = [2; 2; 0; 0]
xₜ, uₜ = Rollout_RK4(point_mass, x₀, xf, H, dt, P, α) #xₜ is [k_steps, 4]
Aₜ = zeros(Float32, (n, n, k_steps))
Bₜ = zeros(Float32, (n, m, k_steps))
Qₜ = zeros(Float32, (n, n, k_steps))
lₜ = zeros(Float32, (n, k_steps))
Rₜ = zeros(Float32, (m, m, k_steps))
rₜ = zeros(Float32, (m, k_steps))

for t = 1:(k_steps)
    # 2. Linearize dynamics about trajectory
    Aₜ[:,:,t], Bₜ[:,:,t] = lin_dyn_discrete(point_mass, xₜ[t,:] - xf, uₜ[t,:], dt)
    # 3. Compute second order Taylor series expansion the cost function
    c, Qₜ[:,:,t], lₜ[:,t], Rₜ[:,:,t], rₜ[:,t] = quadratic_cost(cost, xₜ[t,:] - xf, uₜ[t,:])
end

# 4. Do lqr
P, α = affinelq!(Aₜ, Bₜ, Qₜ, lₜ, Rₜ , rₜ)

size(P)
#end

(2, 4, 3000)

In [148]:
xₜ, uₜ = Rollout_RK4(point_mass, x₀, xf, H, dt, P, α)

([0.0 0.0 0.0 0.0; 9.660896017670631e-5 9.660896017670631e-5 0.019130426862566472 0.019130426862566472; … ; 3.333526516700677 3.333526516700677 0.0004373745817753235 0.0004373745817753235; 3.3335307625407467 3.3335307625407467 0.0004120467906871093 0.0004120467906871093], [3.942016124725342 3.942016124725342; 3.9379452432799242 3.9379452432799242; … ; 2.9449865860996246e-5 2.9449865860996246e-5; 0.0 0.0])

In [150]:
xₜ[end,:]

4-element Vector{Float64}:
 3.3335307625407467
 3.3335307625407467
 0.0004120467906871093
 0.0004120467906871093